In [1]:
# reference: authoring_multibody_simulation.ipynb

# Import some basic libraries and functions for this tutorial.
import numpy as np
import os

from pydrake.common import FindResourceOrThrow, temp_directory
from pydrake.geometry import (
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Role,
    StartMeshcat,
)
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph, MultibodyPlant
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import ModelVisualizer


In [2]:
# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
# import the external motoman URDF model, and visualize it

# First we'll locate one of Drake's example model files, a KUKA iiwa arm.
# Note that FindResourceOrThrow() is only used for models included with Drake.
# Don't use FindResourceOrThrow for your own models.
# iiwa7_model_file = FindResourceOrThrow(
#     "drake/manipulation/models/"
#     "iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")
motoman_model_file = '/home/yinglong/Documents/research/task_motion_planning/infrastructure/motoman_ws/src/pracsys-motoman/motoman/motoman_config/updated_obj_urdf/motoman_dual.urdf'


# Create a model visualizer and add the robot arm.
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().package_map().PopulateFromFolder('/home/yinglong/Documents/research/task_motion_planning/infrastructure/motoman_ws/src/pracsys-motoman/motoman/motoman_config')
visualizer.AddModels(motoman_model_file)

# When this notebook is run in test mode it needs to stop execution without
# user interaction. For interactive model visualization you won't normally
# need the 'loop_once' flag.
test_mode = True if "TEST_SRCDIR" in os.environ else False

# Start the interactive visualizer.
# Click the "Stop Running" button in MeshCat when you're finished.
visualizer.Run(loop_once=test_mode)

Click 'Stop Running' or press Esc to quit


In [3]:
import json
from pydrake.geometry import Box, HalfSpace
from pydrake.multibody.tree import SpatialInertia
from pydrake.multibody.plant import CoulombFriction
import transformations as tf
motoman_model_file = '/home/yinglong/Documents/research/task_motion_planning/infrastructure/motoman_ws/src/pracsys-motoman/motoman/motoman_config/updated_obj_urdf/motoman_dual.urdf'


In [4]:
# load the scene file from json, and construct the workspace

def load_scene_workspace(plant: MultibodyPlant):
    scene_f = 'scene.json'
    f = open(scene_f, 'r')
    scene_dict = json.load(f)
    base_pos = scene_dict['workspace']['pos']
    components = scene_dict['workspace']['components']
    spatial_inertia = SpatialInertia()
    new_model_instance = plant.AddModelInstance("workspace_model_instance")
    body = plant.AddRigidBody(name="workspace", model_instance=new_model_instance, M_BBo_B=spatial_inertia)  # this is the body representing the environment
        
    for component_name, component in components.items():
        print('component name: ')
        print(component_name)
        shape = component['shape']
        shape = np.array(shape)
        # pos = np.array(component['pose']['pos'])
        component['pose']['pos'] = np.array(component['pose']['pos']) + np.array(base_pos)
        pos = np.array(component['pose']['pos'])
        ori = component['pose']['ori']  # x y z w
        alpha = 1.0
        quat = tf.quaternion_matrix([ori[3],ori[0],ori[1],ori[2]])
        R = RotationMatrix(quat[:3,:3])
        body_X_BG = RigidTransform(R=R, p=pos)
        body_friction = CoulombFriction(static_friction=0.6,
                                        dynamic_friction=0.5)
        # create a plant body
        col_shape = Box(width=shape[0], depth=shape[1], height=shape[2])
        vid = plant.RegisterVisualGeometry(body=body, X_BG=body_X_BG, shape=col_shape, name=component_name+'_visual',
                                     diffuse_color=[1., 0.64, 0.0, 1.0])
        cid = plant.RegisterCollisionGeometry(body=body, X_BG=body_X_BG, shape=col_shape, name=component_name+'_collision',
                                        coulomb_friction=body_friction)
        print('vid: ', vid)
        print('cid: ', cid)
    return body, base_pos

In [5]:
# https://www.kwesirutledge.info/thoughts/drake1
def AddGround(plant):
    """
    Add a flat ground with friction
    """

    # Constants
    transparent_color = np.array([0.5,0.5,0.5,0])
    nontransparent_color = np.array([0.5,0.5,0.5,0.1])

    p_GroundOrigin = [0, 0.0, 0.0]
    R_GroundOrigin = RotationMatrix.MakeXRotation(0.0)
    X_GroundOrigin = RigidTransform(R_GroundOrigin,p_GroundOrigin)

    # Set Up Ground on Plant

    surface_friction = CoulombFriction(
            static_friction = 0.7,
            dynamic_friction = 0.5)
    plant.RegisterCollisionGeometry(
            plant.world_body(),
            X_GroundOrigin,
            HalfSpace(),
            "ground_collision",
            surface_friction)
    plant.RegisterVisualGeometry(
            plant.world_body(),
            X_GroundOrigin,
            HalfSpace(),
            "ground_visual",
            transparent_color)  # transparent


In [6]:
def create_scene(sim_time_step=0.0001):
    # Clean up the Meshcat instance.
    meshcat.Delete()
    meshcat.DeleteAddedControls()

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(
        builder, time_step=sim_time_step)
    parser = Parser(plant)

    # Loading models.

    # Weld the table to the world so that it's fixed during the simulation.
    
    parser.package_map().PopulateFromFolder('/home/yinglong/Documents/research/task_motion_planning/infrastructure/motoman_ws/src/pracsys-motoman/motoman/motoman_config')
    parser.AddModels(motoman_model_file)

    table_frame = plant.GetFrameByName("base")
    plant.WeldFrames(plant.world_frame(), table_frame)

    # add workspace
    workspace_body, workspace_pos = load_scene_workspace(plant)
    X_WorldB = RigidTransform(p=workspace_pos)
    print('X_WorldB: ')
    print(X_WorldB)
    plant.WeldFrames(plant.world_frame(), workspace_body.body_frame(), X_WorldB)

    
#     AddGround(plant)
    print('workspace_pos: ', workspace_pos)

    # Finalize the plant after loading the scene.
    plant.Finalize()
    
    print('number of collision geometries: ', plant.num_collision_geometries())
    # We use the default context to calculate the transformation of the table
    # in world frame but this is NOT the context the Diagram consumes.
    plant_context = plant.CreateDefaultContext()

    # Set the initial pose for the free bodies, i.e., the custom box and the
    # cracker box.
    body = plant.GetBodyByName("workspace")

    
    # get the workspace pose    
#     X_WorldB = RigidTransform(p=workspace_pos)

#     plant.SetFreeBodyPose(plant_context, body, X_WorldB)

    # Add visualizer to visualize the geometries.
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat,
        MeshcatVisualizerParams(role=Role.kPerception, prefix="visual"))

    diagram = builder.Build()
    return diagram, visualizer

In [7]:
def initialize_simulation(diagram):
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1.)
    return simulator

def run_simulation(sim_time_step):
    diagram, visualizer = create_scene(sim_time_step)
    simulator = initialize_simulation(diagram)
    visualizer.StartRecording()
    simulator.AdvanceTo(1.0)
    visualizer.PublishRecording()

# Run the simulation with a small time step. Try gradually increasing it!
run_simulation(sim_time_step=0.0001)

component name: 
table
vid:  <GeometryId value=226>
cid:  <GeometryId value=228>
X_WorldB: 
RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[-0.05, 0.0, 0.0],
)
workspace_pos:  [-0.05, 0, 0]
number of collision geometries:  30


In [ ]:


diagram_context = diagram.CreateDefaultContext()
plant.SetFreeBodyPose(
plant.GetMyContextFromRoot(diagram_context),
plant.GetBodyByName("body", block_as_model),
X_WBlock)